## DATAFRAME IMPLEMENTATION

In [0]:
## This script will automaticall fetch and execute the variables and functions from another notebook
"""
%run
./Thavaseelan_Mohan_rdd
"""

Out[2]: '\n%run\n./Thavaseelan_Mohan_rdd\n'

Import required libraries and preprocessing steps

In [0]:
from pyspark.sql.types import *
import re
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Source_file1='clinicaltrial_2023'
Source_file2='pharma'
Source_zip_format_extension='.zip'
Source_file_format='.csv'

#def split_file():
delimiters1=["\t"]
df_clinical=spark.read.csv("/FileStore/tables/" + Source_file1 + Source_file_format,
                           sep=delimiters1,
                           header=True,
                           inferSchema=True)
    
rdd_clinical=df_clinical.rdd

print(f"{Source_file1} loaded successfully!!!")
    
df_Pharma=spark.read.csv("/FileStore/tables/" + Source_file2 + Source_file_format,
                         sep=',',
                         header=True,
                         inferSchema=True)
df_Pharma_rdd=df_Pharma.rdd
print(f"{Source_file2} loaded successfully!!!")
#split_file()

#def create_dataframe():
Schema = StructType([
        StructField("Id", StringType()),
        StructField("StudyTitle", StringType()),
        StructField("Acronym", StringType()),
        StructField("Status", StringType()),
        StructField("Conditions", StringType()),
        StructField("Interventions", StringType()),
        StructField("Sponsor", StringType()),
        StructField("Collaborators", StringType()),
        StructField("Enrollment", StringType()),
        StructField("FunderType", StringType()),
        StructField("Type", StringType()),
        StructField("StudyDesign", StringType()),
        StructField("Start", StringType()),
        StructField("Completion", StringType())
        ])

RDD_raw = rdd_clinical.map(lambda x : (x[0].split('\t')))\
                     .map(lambda x : (x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[-7],x[-6],x[-5],x[-4],x[-3],x[-2],x[-1]))\
                     .collect()
print("Source data converted to RDD Successfully!!!")

Source_Raw = spark.createDataFrame(RDD_raw, Schema)
Raw_file=sc.parallelize(RDD_raw)
df_raw=Raw_file.toDF()
print("Dataframe created for source file")

RDD_cleansed = rdd_clinical.map(lambda x : (x[0].replace('\"','').replace(',','').split('\t')))\
                           .filter(lambda x : x[0] is not None)\
                           .filter(lambda x : x[0]!='')\
                           .filter(lambda x : (x[0]!='NCT00146315'))\
                           .filter(lambda x : len(x[0])<=11)\
                           .map(lambda x : (x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8],x[9],x[10],x[11],x[12],x[13]))\
                           .collect()
print("Data Splitted and Cleansing done Successfully!!!")
Source_Transform = spark.createDataFrame(RDD_cleansed, Schema)
Transf=sc.parallelize(RDD_cleansed)
df_main=Transf.toDF()
print("Dataframe created for cleansed file")

Final_rdd=df_main.rdd
print("RDD for clinical cleansed data created successfully!!!")

clinicaltrial_2023 loaded successfully!!!
pharma loaded successfully!!!
Source data converted to RDD Successfully!!!
Dataframe created for source file
Data Splitted and Cleansing done Successfully!!!
Dataframe created for cleansed file
RDD for clinical cleansed data created successfully!!!


Scenario 1 - The number of studies in the dataset. You must ensure that you explicitly check distinct studies.

In [0]:
def DataFrame_SC1_Distinct_Studies():
    
    Studies_count=Source_Transform.select("Id").dropDuplicates().distinct().count()
                                   
    print("Total no of distinct studies count is:",  Studies_count)

DataFrame_SC1_Distinct_Studies()

Total no of distinct studies count is: 483420


Scenario 2 - You should list all the types (as contained in the Type column) of studies in the dataset along with the frequencies of each type. These should be ordered from most frequent to least frequent.

In [0]:
def DataFrame_SC2_StudyType_Groupwise():

    StudyType_Group = Source_Transform.select("Id","Type")\
                                      .groupBy(trim(("Type")).alias("Type"))\
                                      .agg(count("Type").alias("Count"))

    StudyType_Group.show(truncate=False)

DataFrame_SC2_StudyType_Groupwise()

+---------------+------+
|Type           |Count |
+---------------+------+
|INTERVENTIONAL |371382|
|OBSERVATIONAL  |110221|
|EXPANDED_ACCESS|928   |
|               |889   |
+---------------+------+



Scenario 3 - The top 5 conditions (from Conditions) with their frequencies.

In [0]:
def DataFrame_SC3_Conditions_Frequencies():

    Conditions_Frequency = Source_Transform.select("Id",explode(split("Conditions","\\|")).alias("Conditions"))\
                                        .filter("Conditions is not null")\
                                        .groupBy(trim(("Conditions")).alias("Conditions_transformed"))\
                                           .agg(count("Conditions").alias("Coun"))\
                                           .sort("Coun")\
    
    Order_Window = Window.orderBy(Conditions_Frequency['Coun'].desc())

    cf=Conditions_Frequency.select('*', dense_rank().over(Order_Window).alias('rank'))\
                           .filter(col('rank') <= 6)

    result = cf.show(truncate=False)

DataFrame_SC3_Conditions_Frequencies()

+----------------------+----+----+
|Conditions_transformed|Coun|rank|
+----------------------+----+----+
|Healthy               |9731|1   |
|Breast Cancer         |7502|2   |
|Obesity               |6549|3   |
|Stroke                |4073|4   |
|Hypertension          |4024|5   |
|Depression            |3911|6   |
+----------------------+----+----+



Scenario 4 - Find the 10 most common sponsors that are not pharmaceutical companies, along with the number of clinical trials they have sponsored.

In [0]:
def DataFrame_SC4_Non_Common_sponsors_Trials():
    
    Required_Fields = Source_Transform.select(trim("Sponsor").alias("Sponsor") ,"Id")\
                                      .distinct()
                                    
    Clincal_data = Source_Transform.select(trim("Sponsor").alias("Sponsor_Clinical"))\
                                   .filter("Sponsor_Clinical is not null")\
                                   .filter(trim("Sponsor_Clinical")!='')\
                                   .distinct()
                                 
    Pharmacy_data = df_Pharma.select(trim("Parent_Company").alias("Parent_Company_Pharma"))\
                             .filter("Parent_Company_Pharma is not null")\
                             .filter(trim("Parent_Company_Pharma")!='')\
                             .distinct()
                           
    Non_Common_Pharamcy = Clincal_data.exceptAll(Pharmacy_data)\
                                      .select(Clincal_data["Sponsor_Clinical"])
                                    
    jn = Required_Fields.join(Non_Common_Pharamcy,Required_Fields["Sponsor"]==Non_Common_Pharamcy["Sponsor_Clinical"],"inner")\
                        .select(Required_Fields["Sponsor"],Required_Fields["Id"])\
                        .groupBy("Sponsor")\
                        .agg(count("Sponsor")\
                        .alias("Coun"))

    Non_Pharma_Trials = jn.sort(jn.Coun.desc())\
                          .show(10,truncate=False)


DataFrame_SC4_Non_Common_sponsors_Trials()

+-------------------------------------------------------------+----+
|Sponsor                                                      |Coun|
+-------------------------------------------------------------+----+
|National Cancer Institute (NCI)                              |3410|
|Assiut University                                            |3335|
|Cairo University                                             |3023|
|Assistance Publique - Hôpitaux de Paris                      |2951|
|Mayo Clinic                                                  |2766|
|M.D. Anderson Cancer Center                                  |2702|
|Novartis Pharmaceuticals                                     |2393|
|National Institute of Allergy and Infectious Diseases (NIAID)|2340|
|Massachusetts General Hospital                               |2263|
|National Taiwan University Hospital                          |2181|
+-------------------------------------------------------------+----+
only showing top 10 rows



Scenario 5 - Plot number of completed studies for each month in 2023. You need to include your visualization as well as a table of all the values you have plotted for each month.

In [0]:
def DataFrame_SC5_Completed_Status_Monthwise():

    Final = Source_Transform.filter(lower("Status")=="completed")\
                            .filter(year(to_date("Completion"))=='2023')\
                            .select("Id","Completion",to_date(trim("Completion")).alias("Date-Format"),year(to_date(trim("Completion"))).alias("Year"),
                                    lpad(month(to_date(trim("Completion"))),2,'0').alias("Month"))\
                            .groupBy(lpad(month(to_date(trim("Completion"))),2,'0').alias("Month"))\
                            .agg(count(lpad(month(to_date(trim("Completion"))),2,'0')).alias("Count"))\
                            .sort("Month").replace('01','Jan').replace('02','Feb').replace('03','Mar').replace('04','Apr').replace('05','May').\
                            replace('06','June').replace('07','July').replace('08','August').replace('09','September').replace('10','October').\
                            replace('11','November').replace('12','December')\
                            .show()

DataFrame_SC5_Completed_Status_Monthwise()

+---------+-----+
|    Month|Count|
+---------+-----+
|      Jan| 1494|
|      Feb| 1272|
|      Mar| 1552|
|      Apr| 1324|
|      May| 1415|
|     June| 1619|
|     July| 1360|
|   August| 1230|
|September| 1152|
|  October| 1058|
| November|  909|
| December| 1082|
+---------+-----+



Additional Negative Scenario - TO find Members with Completion status with future datespan

In [0]:
def DataFrame_Addition_Negative_Scenario():

    Final = Source_Transform.filter(lower("Status")=="completed")\
                            .filter(year(to_date(trim("Completion")))>'2023')\
                            .select("Id","Status","Completion",to_date("Completion").alias("Date-Format"),year(to_date("Completion")).alias("Year")).groupBy("Year","Status")\
                            .agg(count("Status").alias("Count"))\
                            .sort("Year")\
                            .display()
                            
DataFrame_Addition_Negative_Scenario()

Year,Status,Count
2024,COMPLETED,622
2031,COMPLETED,1


In [0]:
Final = Source_Transform.filter(lower("Status")=="completed")\
                            .filter(year(to_date(trim("Completion")))>'2023')\
                            .select("Id","Status","Conditions","Completion",to_date("Completion").alias("Date-Format"),year(to_date("Completion")).alias("Year"))\
                            .sort("Completion",ascending=False)\
                            .show(truncate=False)

+-----------+---------+-------------------------------------------------------------------------+----------+-----------+----+
|Id         |Status   |Conditions                                                               |Completion|Date-Format|Year|
+-----------+---------+-------------------------------------------------------------------------+----------+-----------+----+
|NCT00888160|COMPLETED|Unstable Pelvic Ring Fracture|Unstable Acetabulum Fracture|Femur Fracture|2031-01   |2031-01-01 |2031|
|NCT06055088|COMPLETED|Breast Cancer|Neuropathy                                                 |2024-02-16|2024-02-16 |2024|
|NCT06129721|COMPLETED|Acute Stroke|Ischemic Stroke Acute|Vertebro Basilar Ischemia             |2024-02-16|2024-02-16 |2024|
|NCT06197659|COMPLETED|Postoperative Nausea and Vomiting                                        |2024-02-15|2024-02-15 |2024|
|NCT06226207|COMPLETED|Reliability and Validity                                                 |2024-02-15|2024-02-15